In [ ]:
import siibra
from nilearn import plotting
from IPython.display import display, Markdown

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such point, amongst other possibilities, by clicking them in the atlas viewer.

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
c1 = "-25.650mm, -2.750mm, -33.750mm"
c2 = "-37.350mm, -81.050mm, -6.300mm"

In [ ]:
points = siibra.PointSet([c1,c2],'mni152')
plotting.view_markers([tuple(p) for p in points], ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS
julich_pmaps = atlas.get_map(
    space="mni152",
    parcellation="julich 2.9",
    maptype="continuous")
assignments = julich_pmaps.assign_coordinates(points,sigma_mm=5)

Let's look at the most probable area found for each point.

In [ ]:
point1, point2 = points
region1, mapindex1, scores1 = assignments[0][0]
region2, mapindex2, scores2 = assignments[1][0]
regionmap1 = julich_pmaps.fetch(mapindex1)
regionmap2 = julich_pmaps.fetch(mapindex2)

view1 = plotting.plot_stat_map( 
    regionmap1, 
    cut_coords=tuple(point1),
    title=f"{region1.name} ({scores1['correlation']:.2f})" 
)
view1.add_markers([tuple(point1)],['cyan'])

view2 = plotting.plot_stat_map( 
    regionmap2, 
    cut_coords=tuple(point2),
    title=f"{region2.name} ({scores2['correlation']:.2f})"
)
view2.add_markers([tuple(point2)],['cyan'])

### Look for white matter bundles connecting the regions

Next, we look at the most probable region associated to each point, considering them as a source and target region to investigate connectivity. In order to find white matter fibre bundles which are likely to connect them, we use the probability maps of the white matter fibre bundle parcellation, and assign both the source and target region from Julich-Brain to the fibre bundles. The intersection of the resulting bundles gives us those that are likely to provide connections from source to target.

In [ ]:
# get the probabilistic maps of long fibre bundles
bundlemaps = atlas.get_map("mni152", "long bundle", "continuous")

# assign the source and target region to the bundles, 
# using their probability maps found above.
bundles1 = bundlemaps.assign(regionmap1, msg=f"Bundles touching {region1.name}")
bundles2 = bundlemaps.assign(regionmap2, msg=f"Bundles touching {region2.name}")
                
# intersect the two sets of related fiber bundles
intersection = {
    (b[0],b[1]) for b in bundles1+bundles2
    if  any(b1[0]==b[0] for b1 in bundles1) 
    and any(b2[0]==b[0] for b2 in bundles2) }

In [ ]:
# plot each of the bundles in the intersection (but no more than 3)
for bundle,bundlemapindex in intersection:
    view=plotting.plot_stat_map(
         bundlemaps.fetch(bundlemapindex),
        title=f"{region1.name} and {region2.name} connected by {bundle.name} ")
    view.add_overlay(regionmap1)
    view.add_overlay(regionmap2)
    view.add_markers([tuple(p) for p in points],['white'])

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the two regions as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# select the source region
with siibra.QUIET:
    profiles = siibra.get_features(region1, siibra.modalities.ConnectivityProfile)
print(f"Found {len(profiles)} different connectivity profiles going out from {region1.name}:")

There are in general multiple features, so let's see which types of connectivity profiles have been found?

In [ ]:
for i,p in enumerate(profiles):
    print(f"{i:3} - {p.name}")

We choose a structural connectivity profile, and decode the names of the connected regions. To retreive deeper metadata for the dataset from the EBRAINS knowledge graph, we need an EBRAINS authentication token.

In [ ]:
with siibra.QUIET:
    siibra.set_ebrains_token(input("Paste EBRAINS auth token:"))

In [ ]:
p = profiles[0]
print(p.name)
display(Markdown(p.description))
with siibra.QUIET:
    connections = p.decode(region1.parcellation)

Create a bar plot of the connectivity profile, and identify the target region from above.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

i = [r for v,r in connections].index(region2)

N = max(20,i)
X = range(N)
Y = [v for v,_ in connections[:N]]
L = [r.name for v,r in connections[:N]]

fig  = plt.figure(figsize=(12,4))
ax1  = fig.add_subplot()
ax1.set_xticks(X)
ax1.set_xticklabels(L,rotation=60,ha='right')
ax1.bar(X,Y,0.8)

# where is region2?
i = [r for v,r in connections].index(region2)
ax1.bar(i,connections[i][0],0.9,color='r')

fig.gca().set_title(f"Connection strengths from area {region1.name}")
plt.show()